# Description

The scripts computes the measures "coverage" and "overflow" to compare computed shots with the ground truth shots. It gives indication about how well the visual change classifier works.

The script takes the visual change dataset and considers the root layer information, only. Each label as "visual change" from one frame to another is considered as shot boundary on the root layer. These shots are called "ground truth shots".

It is also created a visual change classifier on basis of the labeling of one participant's session and computes "visual change" labels on all participants' sessions on one Web site. These labels are also considered for the root layer, only, and used again as shot boundaries. These shots are called "computed shots".

"Coverage" and "overflow" measures are inspired from:

`J. Vendrig and M. Worring, "Systematic evaluation of logical story unit segmentation," in IEEE Transactions on Multimedia, vol. 4, no. 4, pp. 492-499, Dec. 2002.
doi: 10.1109/TMM.2002.802021
`

### Imports

In [ ]:
# Modules
import pandas as pd
import numpy as np
import os.path
from collections import defaultdict

# Custom modules
from classifier import Classifier

# Settings

In [ ]:
# Settings
min_obs_extent = 32 # extent too which obs are withdrawn
baseline_feature_name = 'pixel_diff_count_bgr'
classifier_names = ['svc', 'forest', 'baseline']

# Defines
dataset_visual_change_dir = r'C:/StimuliDiscoveryData/Dataset_visual_change' # SET ME!
participants = ['p1', 'p2', 'p3', 'p4']
training_participants = ['p1']
exclude_training_from_test = False

# Categories
shopping = ['walmart', 'amazon', 'steam']
news = ['reddit', 'cnn', 'guardian']
health = ['nih', 'webmd', 'mayo']
cars = ['gm', 'nissan', 'kia']
categories = {'shopping': shopping, 'news': news, 'health': health, 'cars': cars}

# Drop and filter features
features_drop = [
    'bag_of_words_vocabulary_size',
    'optical_flow_angle_min',
    'optical_flow_angle_max',
    'optical_flow_magnitude_min']
features_filter = [ # not applied if empty (all features but dropped ones are then considered)
    'edge_change_fraction',
    'mssim_b',
    'mssim_g',
    'mssim_r',
    'pixel_diff_acc_b',
    'pixel_diff_acc_bgr',
    'pixel_diff_acc_g',
    'pixel_diff_acc_gray',
    'pixel_diff_acc_hue',
    'pixel_diff_acc_lightness',
    'pixel_diff_acc_r',
    'pixel_diff_acc_saturation',
    'pixel_diff_count_b',
    'pixel_diff_count_bgr',
    'pixel_diff_count_g',
    'pixel_diff_count_gray',
    'pixel_diff_count_hue',
    'pixel_diff_count_lightness',
    'pixel_diff_count_r',
    'pixel_diff_count_saturation',
    'psnr',
    'sift_match',
    'sift_match_0',
    'sift_match_16',
    'sift_match_256',
    'sift_match_4',
    'sift_match_512',
    'sift_match_64',
    'sift_match_distance_max',
    'sift_match_distance_mean',
    'sift_match_distance_min',
    'sift_match_distance_stddev',
    'sift_match_spatial'
]

# Session

In [ ]:
# Session class holds one site visit by a participant
class Session:
    
    # Constructor
    def __init__(self, participant, site):
        
        # Store some members of general interest
        self.p = participant
        self.s = site
        
        # Load dataset (header of features dataset has extra comma)
        self.f_df = pd.read_csv(dataset_visual_change_dir + '/' + participant + '/' + site + '_features.csv')
        self.mf_df = pd.read_csv(dataset_visual_change_dir + '/' + participant + '/' + site + '_features_meta.csv')
        self.l1_df = pd.read_csv(dataset_visual_change_dir + '/' + participant + '/' + site + '_labels-l1.csv', header=None, names=['label'])
        self.m_df = pd.read_csv(dataset_visual_change_dir + '/' + participant + '/' + site + '_meta.csv')
        
        # Drop columns of non-interest
        self.f_df = self.f_df.drop(features_drop, axis=1)
        
        # Filter for columns of interest
        if len(features_filter) > 0:
            self.f_df = self.f_df.filter(items=features_filter, axis=1)
        
        # Drop observations that are smaller than a certain extent
        width_idxs = self.mf_df[self.mf_df['overlap_width'] <= min_obs_extent].index
        height_idxs = self.mf_df[self.mf_df['overlap_height'] <= min_obs_extent].index
        drop_idxs = list(set(width_idxs) | set(height_idxs))
        self.f_df = self.f_df.drop(drop_idxs, axis=0)
        self.mf_df = self.mf_df.drop(drop_idxs, axis=0)
        self.l1_df = self.l1_df.drop(drop_idxs, axis=0)
        
        # Replace some values
        if 'optical_flow_magnitude_max' in self.f_df.columns:

            # Replace infinity datapoints in 'optical_flow_magnitude_max' with maximum value (encoded as -1)
            max_value = self.f_df['optical_flow_magnitude_max'].max() # maximum from complete training data

            # In both, training an test data
            self.f_df[self.f_df['optical_flow_magnitude_max'] == -1] = max_value

# Compute shots

In [ ]:
def compute_shots(df, frame_count):

    # Filter for layer ('root') and label 1
    df = df.loc[df['layer_type'] == 'root']
    df = df.loc[df['label'] == 1]

    # Shots
    end_frames = list(df['prev_video_frame'])
    end_frames.append(frame_count-1)
    shots = []
    for i in range(len(end_frames)):
        start_frame = 0
        prev_i = i - 1
        if prev_i >= 0:
            start_frame = end_frames[prev_i] + 1
        end_frame = end_frames[i]
        shots.append((start_frame, end_frame))
    return shots

# Compute coverage

In [ ]:
def compute_coverage(gt_shots, c_shots):

    overall_cover = 0.0

    # Go over ground truth shots
    for (gt_start, gt_end) in gt_shots:

        shot_length = gt_end - gt_start + 1
        max_cover = 0
        gt_range = range(gt_start, gt_end+1)

        # Go over computed shots and find one with maximum coverage
        for (c_start, c_end) in c_shots:
            c_range = range(c_start, c_end+1)
            cover = len(list(set(gt_range) & set(c_range)))
            if cover > max_cover:
                max_cover = cover

        # Compute coverage
        cover = float(max_cover) / float(shot_length)
        overall_cover += cover * (shot_length / frame_count)

    return overall_cover

# Compute overflow

In [ ]:
def compute_overflow(gt_shots, c_shots):

    overall_over = 0.0

    # Go over ground truth shots
    for i in range(len(gt_shots)):

        (gt_start, gt_end) = gt_shots[i]
        shot_length = gt_end - gt_start + 1
        gt_range = range(gt_start, gt_end+1)

        # Get previous ground truth shot
        prev_gt_range = []
        prev_gt_i = i-1
        if prev_gt_i >= 0:
            (prev_gt_start, prev_gt_end) = gt_shots[prev_gt_i]
            prev_gt_range = range(prev_gt_start, prev_gt_end+1)

        # Get next ground truth shot
        next_gt_range = []
        next_gt_i = i+1
        if next_gt_i < len(gt_shots):
            (next_gt_start, next_gt_end) = gt_shots[next_gt_i]
            next_gt_range = range(next_gt_start, next_gt_end+1)

        denom = float(len(prev_gt_range) + len(next_gt_range))
        nom = 0.0

        # Go over computed shots
        for (c_start, c_end) in c_shots:
            c_range = range(c_start, c_end+1)

            # Check for intersection between frames from ground truth shot and computed shot
            intersection_gt = len(list(set(gt_range) & set(c_range)))
            intersection_gt_at_all = min(intersection_gt, 1) # limit to 0 or 1
            
            # Count only frames from computed shot that intersect (aka overflow...) with prev or next ground truth shot
            intersection_gt_prev = len(list(set(prev_gt_range) & set(c_range)))
            intersection_gt_next = len(list(set(next_gt_range) & set(c_range)))
            
            # Compute potential overflow
            nom += max(intersection_gt_prev, intersection_gt_next) * intersection_gt_at_all

        # Compute overflow
        over = nom/denom
        overall_over += over * (shot_length / frame_count)
    
    return overall_over

# Implementation

In [ ]:
sites_measures = {} # for each site, holds a dict mapping from training session to tuple of coverage and overflow
for name, sites in categories.items():
    for site in sites:
        
        print('Site: ' + site)
        
        measures_per_training = {} # maps from training session to tuple of coverage and overflow
        
        # Go over participants chosen for training
        for training_p in training_participants:
            
            print('Training with: ' + training_p)
            coverages = defaultdict(list)
            overflows = defaultdict(list)

            # Create training data
            training_session = Session(training_p, site)
            X_train = training_session.f_df.values
            y_train = training_session.l1_df.values.flatten()
            idx_baseline = training_session.f_df.columns.get_loc(baseline_feature_name)

            # Go over participants that are not used for training
            test_participants = list(participants)
            if exclude_training_from_test:
                test_participants.remove(training_p)
            for p in test_participants:

                # Load session to work on
                session = Session(p, site)
                frame_count = int(session.m_df['screencast_frame_total_count'])

                # Create classifier
                classifier = Classifier()

                # Create dataframes with ground truth labels and the computed labels
                gt_df = pd.concat([session.mf_df, session.l1_df], axis=1)
                c_df = pd.concat([session.mf_df, session.l1_df], axis=1)

                # Apply classifier to create computed labels
                pred = classifier.apply(X_train, y_train, session.f_df.values, idx_baseline)

                # Go over available classifiers and perform computation for each
                for name in classifier_names:
                    c_df['label'] = pred[name] # overwrite labels for computed shots

                    # Compute shots
                    gt_shots = compute_shots(gt_df, frame_count)
                    c_shots = compute_shots(c_df, frame_count)

                    # Compute measurements
                    coverage = compute_coverage(gt_shots, c_shots)
                    overflow = compute_overflow(gt_shots, c_shots)
                    coverages[name].append(coverage)# dict that maps classifier name to coverage
                    overflows[name].append(overflow)# dict that maps classifier name to overflow

                    # Output information
                    '''
                    print(
                        p + ':'
                        + ' coverage = ' + '{:1.2f}'.format(coverage)
                        + ', overflow = ' + '{:1.2f}'.format(overflow)
                        + ', gt_shots = ' + '{:3}'.format(len(gt_shots))
                        + ', c_shots = '  + '{:3}'.format(len(c_shots)))
                    '''

            # Store results for one session as training
            measures_per_training[training_p] = (coverages, overflows)
            
            '''
            print(
            '$' + '{:1.2f}'.format(np.mean(coverages['svc'])) + '\\pm' + '{:1.2f}'.format(np.std(coverages['svc']) + '$')
            + ', overflow = ' + '{:1.2f}'.format(np.mean(overflows['svc'])) + '\\pm' + '{:1.2f}'.format(np.std(overflows['svc'])))
            '''
        
        # Store results across all sessions as training
        sites_measures[site] = measures_per_training  

# Print sites
for site, _ in sites_measures.items():
    print(site, end=' ')
print()

print('Coverage')
    
# Print coverages
for _, measures_per_training in sites_measures.items(): # go over sites
    svc_means = []
    forest_means = []
    baseline_means = []
    for training_p in training_participants:
        svc_means.append(np.mean(measures_per_training[training_p][0]['svc']))
        forest_means.append(np.mean(measures_per_training[training_p][0]['forest']))
        baseline_means.append(np.mean(measures_per_training[training_p][0]['baseline']))
        
    print(
        '$' + '{:1.2f}'.format(np.mean(svc_means)) + '\\pm' + '{:1.2f}'.format(np.std(svc_means)) + '$'
        + ' & '
        + '$' + '{:1.2f}'.format(np.mean(forest_means)) + '\\pm' + '{:1.2f}'.format(np.std(forest_means)) + '$'
        + ' & '
        + '$' + '{:1.2f}'.format(np.mean(baseline_means)) + '\\pm' + '{:1.2f}'.format(np.std(baseline_means)) + '$'
        + ' & ')
print()

print('Overflow')

# Print overflows
for _, measures_per_training in sites_measures.items(): # go over sites
    svc_means = []
    forest_means = []
    baseline_means = []
    for training_p in training_participants:
        svc_means.append(np.mean(measures_per_training[training_p][1]['svc']))
        forest_means.append(np.mean(measures_per_training[training_p][1]['forest']))
        baseline_means.append(np.mean(measures_per_training[training_p][1]['baseline']))
        
    print(
        '$' + '{:1.2f}'.format(np.mean(svc_means)) + '\\pm' + '{:1.2f}'.format(np.std(svc_means)) + '$'
        + ' & '
        + '$' + '{:1.2f}'.format(np.mean(forest_means)) + '\\pm' + '{:1.2f}'.format(np.std(forest_means)) + '$'
        + ' & '
        + '$' + '{:1.2f}'.format(np.mean(baseline_means)) + '\\pm' + '{:1.2f}'.format(np.std(baseline_means)) + '$'
        + ' & ')